In [11]:
# Load the Drive helper and mount
from google.colab import drive

# This will proot for authorization
drive.mount('/content/drive')

import pandas as pd
import os

SEQ_LEN = 60 # how long of a proceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3 # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
  if float(future) > float(current):
    return 1
  else:
    return 0

# take featuresets and combine them into sequences of 60 of these featuresets

main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"] # the 4 ratios we want to consider
for ratio in ratios: # begin iteration
  dataset = f"/content/drive/My Drive/Colab Notebooks/crypto/crypto_data/{ratio}.csv" # get full path to the file
  
  df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"]) # read in specific file
  
  # renaming volume and close to include the ticker so we can see which close/volume is which
  df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
  
  df.set_index("time", inplace=True) # set time as index so we can join them on this shared time
  df = df[[f"{ratio}_close", f"{ratio}_volume"]] # ignore the other columns besides price and volume
  
  if len(main_df) == 0: # if the dataframe is empty
    main_df = df # then it's just the
  else: # otherwise, join this data to the main one
    main_df = main_df.join(df)
  
main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

main_df["target"] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head())

Mounted at /content/drive
            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
